In [1]:
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
from Curves import Curves
from ImportData import importSWEiopa
from BondClasses import CorporateBond,CorporateBondPriced

## Input files

There are multiple input files needed to calibrate the fixed income portfolio. They are located in the "Input" folder.

### Parameters.csv

Parameters file holds information about the type of run and the modelling date.

 - EIOPA_param_file ...the relative location of the EIOPA parameter file that will be used as the RFR Ex. "Input/Param_no_VA.csv"
 - EIOPA_curves_file ... the relative location of the EIOPA yield curve that will be used as the RFR Ex. "Input/Curves_no_VA.csv"
 - country ... the name of the country that will be used as the base for this run Ex. "Slovenia"
 - n_proj_years ... length of a run in years starting from the Modelling date Ex. 50
 - Precision ... precision parameter specifying the acceptable tollerance between the calibrated bond price and the market value Ex. 0.00000001
 - Tau ... the acceptable size of the gap between the extrapolated yield rate and the ulitmate forward rate Ex. 0.0001
 - compounding ... the way that the interest rates are compounded in the run Ex. -1
 - Modelling_Date ... the starting date of the run specified as a date string Ex."29/04/2023"


### EIOPA RFR files

There are two types of files derived from the monthly EIOPA RFR submision that are used in this model. The "Curves_XX.csv" containing the yearly yield curves for all countries in scope and the "Param_XX.csv" with the paameters used to derive the curves. These files are used to derive the risk free term structure at the modelling date and to efficiently project the evolution of the term structure.

### Portfolio description

The modelled portfolio is split by asset classes. The fixed income portfolio is located in the file "Bond_Portfolio.csv". Each security needs the following fields:

 -  Asset ID ... unique id such as an ISIN, SEDOL or CUSIP code Ex. IT1234567891
 -  Asset_Type ... asset type string Ex. "Corporate_Bond"
 -  NACE ... NACE asset classification code (nomenclature statistique des activités économiques dans la Communauté européenne) Ex. A1.4.5
 -  Issue_Date ... the string date specifying the issue date of the bond Ex. 3/12/2021
 -  Maturity_Date ... the string date specifying the maturity date of the bond Ex. 3/12/2021
 -  Notional_amount ... the notional amount of the bond Ex. 100
 -  Coupon_Rate ... percentage of the notional amount paid in dividends every period (specified by Frequency) Ex. 0.0014
 -  Frequency ... number of times per a year that dividends are paid Ex. 1 (once per a year)
 -  Recovery_Rate ... percentage of the notional amound that can be recovered in case of a default Ex. 0.80
 -  Default_Probability ... percentage probability of default per year Ex. 0.012
 -  Market_Price ... market price of the bond at the modelling date Ex. 96

### Sector spread
The list of NACE sector codes and the sector specific spread over the risk free rate
 - NACE ... NACE code of the issuer Ex. "A1.1" 
 - NACE code text  ... description of the NACE code for this issuer Ex. "Growing of non-perennial crops" 
 - sSpread  ...  NACE sector specific spread over the risk free rate Ex. 0.01


In [3]:
paramfile = pd.read_csv("Input/Parameters.csv")
paramfile.index = paramfile["Parameter"]
del paramfile["Parameter"]

In [4]:
paramfile

,value
Parameter,
EIOPA_param_file,Input/Param_no_VA.csv
EIOPA_curves_file,Input/Curves_no_VA.csv
country,Slovenia
run_type,Risk Neutral
n_proj_years,50
Precision,1E-10
Tau,0.0001
compounding,-1
Modelling_Date,29/04/2023


In [5]:
selected_param_file = paramfile.loc["EIOPA_param_file"][0]
selected_curves_file = paramfile.loc["EIOPA_curves_file"][0]
country = paramfile.loc["country"]["value"]
run_type = paramfile.loc["run_type"]["value"]
compounding = int(paramfile.loc["compounding"]["value"])

MD = dt.datetime.strptime(paramfile.loc["Modelling_Date"]["value"],"%d/%m/%Y")
MD = dt.date(MD.year,MD.month,MD.day)

In [6]:
Precision = float(paramfile.loc["Precision"][0]) # Numeric precision of the optimisation
Tau = float(paramfile.loc["Tau"][0]) # Targeted distance between the extrapolated curve and the ultimate forward rate at the convergence point

In [7]:
spreadfile = pd.read_csv("Input/Sector_Spread.csv")
spreadfile.index = spreadfile["NACE"]
del spreadfile["NACE"]

In [8]:
bondfile = pd.read_csv("Input/Bond_Portfolio.csv")
bondfile.index = bondfile["Asset ID"]
del bondfile["Asset ID"]

# IMPORT EIOPA CURVE

From the EIOPA files, import the selected curve. The selection consists of what country and what type of curve (Example with or without Volatility Adjustment)

In [9]:
[maturities_country, curve_country, extra_param, Qb]= importSWEiopa(selected_param_file, selected_curves_file, country)

In [10]:
# Maturity of observations:
M_Obs = np.transpose(np.array(maturities_country.values))

# Ultimate froward rate ufr represents the rate to which the rate curve will converge as time increases:
ufr = extra_param.iloc[3]/100

# Convergence speed parameter alpha controls the speed at which the curve converges towards the ufr from the last liquid point:
alpha = extra_param.iloc[4]

# Qb calibration vector published by EIOPA for the curve calibration:
Qb = np.transpose(np.array(Qb.values))

Save the calibration parameters of the selected curve into the Curves instance:

In [11]:
Curves = Curves(ufr, Precision, Tau, MD, country)

# PRICING OF A COUPON BOND

In [12]:
display(bondfile)

,Asset_Type,NACE,Issue_Date,Maturity_Date,Notional_amount,Coupon_Rate,Frequency,Recovery_Rate,Default_Probability,Market_Price
Asset ID,,,,,,,,,,
1,Corporate_Bond,A1.4.5,3/12/2021,12/12/2026,100,0.03,1,0.4,0.03,94
2,Corporate_Bond,B5.2.0,3/12/2021,12/12/2028,100,0.05,1,0.4,0.03,92
3,Corporate_Bond,B8.9.3,3/12/2019,3/12/2020,100,0.04,1,0.4,0.03,96


In [13]:
def converttime(date_time):
    format = "%d/%m/%Y"
    datetime_str = dt.datetime.strptime(date_time,format).date()
    return datetime_str

In [14]:
nace = np.array([])
issuedate = np.array([])
maturitydate = np.array([])
notional = np.array([])
couponrate = np.array([])
frequency = np.array([])
recovrate = np.array([])
defprob = np.array([])
sspread = np.array([])
zspread = np.array([])
marketprice = np.array([])

for iCount in range(0,bondfile.index.size):
    nace = np.append(nace,bondfile["NACE"].iloc[iCount])
    issuedate = np.append(issuedate,converttime(bondfile["Issue_Date"].iloc[iCount]))
    maturitydate = np.append(maturitydate,converttime(bondfile["Maturity_Date"].iloc[iCount]))
    notional = np.append(notional,bondfile["Notional_amount"].iloc[iCount])
    couponrate = np.append(couponrate,bondfile["Coupon_Rate"].iloc[iCount])
    frequency = np.append(frequency,bondfile["Frequency"].iloc[iCount])
    recovrate = np.append(recovrate,bondfile["Recovery_Rate"].iloc[iCount])
    defprob = np.append(defprob,bondfile["Default_Probability"].iloc[iCount])
    if run_type=="Risk Neutral":
        sspread = np.append(sspread,0)
    else:
        sspread = np.append(sspread,spreadfile.loc[bondfile["NACE"].iloc[iCount]]["sSpread"])   
    marketprice = np.append(marketprice, bondfile["Market_Price"].iloc[iCount])

In [15]:
cb = CorporateBond(issuedate, maturitydate, frequency, notional, couponrate, recovrate, defprob, sspread, zspread, marketprice,compounding)

### createcashflowdates() function

Based on the bond description, issue and maturity date, this function creates a list of lists of dates when the cash flows are expected to be paid. 

Note that this list is indepenedent of the modelling date.

In [16]:
cb.createcashflowdates()

Test to see what the function produced:

In [17]:
cb.coupondates

[array([datetime.date(2022, 12, 3), datetime.date(2023, 12, 3),
        datetime.date(2024, 12, 3), datetime.date(2025, 12, 3),
        datetime.date(2026, 12, 3)], dtype=object),
 array([datetime.date(2022, 12, 3), datetime.date(2023, 12, 3),
        datetime.date(2024, 12, 3), datetime.date(2025, 12, 3),
        datetime.date(2026, 12, 3), datetime.date(2027, 12, 3),
        datetime.date(2028, 12, 3)], dtype=object),
 array([datetime.date(2020, 12, 3)], dtype=object)]

In [18]:
cb.notionaldates

[array([datetime.date(2026, 12, 12)], dtype=object),
 array([datetime.date(2028, 12, 12)], dtype=object),
 array([datetime.date(2020, 12, 3)], dtype=object)]

In [19]:
#cb.createcashflows()

### refactordates() function

Given a specific modelling date, the function calculates the fractionns of the year between the modelling date and each cash flow. Only cash flows after the modelling date are considered. The second input is the index number of cashflows that occur after the modelling date.


In [20]:
cb.refactordates(MD)

[[array([0.59685147, 1.59890486, 2.5982204 , 3.59753593]),
  array([0.59685147, 1.59890486, 2.5982204 , 3.59753593, 4.59685147,
         5.59890486]),
  array([], dtype=float64)],
 [array([1, 2, 3, 4]), array([1, 2, 3, 4, 5, 6]), array([], dtype=int32)],
 [array([3.62217659]), array([5.62354552]), array([], dtype=float64)],
 [array([1]), array([1]), array([], dtype=int32)]]

Test refactordates() function

In [21]:
cb.coupondatesfrac

[array([0.59685147, 1.59890486, 2.5982204 , 3.59753593]),
 array([0.59685147, 1.59890486, 2.5982204 , 3.59753593, 4.59685147,
        5.59890486]),
 array([], dtype=float64)]

### generatecashflows()

Generates the vector of cash flows for each asset. One for each coupon and one for each notional amount.

In [26]:
cb.createcashflowsnew()

[[array([3., 3., 3., 3.]),
  array([5., 5., 5., 5., 5., 5.]),
  array([], dtype=float64)],
 [array([100.]), array([100.]), array([100.])]]

### Calibrate the bonds

In [ ]:
def BisectionsSpread(obj, xStart, xEnd, coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread, marketprice, Precision, maxIter):
    """
    Bisection root finding algorithm for finding the root of a function. The function here is the allowed difference between the ultimate forward rate and the extrapolated curve using Smith & Wilson.

    Args:
        cbPriced =  CorporateBondPriced object containing the list of priced bonds, spreads and cash flows
        xStart =    1 x 1 floating number representing the minimum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.05
        xEnd =      1 x 1 floating number representing the maximum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.8
        M_Obs =     n x 1 ndarray of maturities of bonds, that have rates provided in input (r). Ex. u = [[1], [3]]
        r_Obs =     n x 1 ndarray of rates, for which you wish to calibrate the algorithm. Each rate belongs to an observable Zero-Coupon Bond with a known maturity. Ex. r = [[0.0024], [0.0034]]
        ufr  =      1 x 1 floating number, representing the ultimate forward rate. Ex. ufr = 0.042
        Tau =       1 x 1 floating number representing the allowed difference between ufr and actual curve. Ex. Tau = 0.00001
        Precision = 1 x 1 floating number representing the precision of the calculation. Higher the precision, more accurate the estimation of the root
        maxIter =   1 x 1 positive integer representing the maximum number of iterations allowed. This is to prevent an infinite loop in case the method does not converge to a solution         
    
    Returns:
        1 x 1 floating number representing the optimal value of the parameter alpha 

    Example of use:
        >>> import numpy as np
        >>> from SWCalibrate import SWCalibrate as SWCalibrate
        >>> M_Obs = np.transpose(np.array([1, 2, 4, 5, 6, 7]))
        >>> r_Obs =  np.transpose(np.array([0.01, 0.02, 0.03, 0.032, 0.035, 0.04]))
        >>> xStart = 0.05
        >>> xEnd = 0.5
        >>> maxIter = 1000
        >>> alfa = 0.15
        >>> ufr = 0.042
        >>> Precision = 0.0000000001
        >>> Tau = 0.0001
        >>> BisectionAlpha(xStart, xEnd, M_Obs, r_Obs, ufr, Tau, Precision, maxIter)
        [Out] 0.11549789285636511

    For more information see https://www.eiopa.europa.eu/sites/default/files/risk_free_interest_rate/12092019-technical_documentation.pdf and https://en.wikipedia.org/wiki/Bisection_method
    
    Implemented by Gregor Fabjan from Qnity Consultants on 17/12/2021.
    """   

    yStart = obj.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xStart)
    yEnd = obj.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xEnd)
    if np.abs(yStart-marketprice) < Precision:
        return xStart
    if np.abs(yEnd-marketprice) < Precision:
        return xEnd # If final point already satisfies the conditions return end point
    iIter = 0
    while iIter <= maxIter:
        xMid = (xEnd+xStart)/2 # calculate mid-point
        yMid = obj.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xMid) # What is the solution at midpoint
        if ((yStart-marketprice) == 0 or (xEnd-xStart)/2 < Precision): # Solution found
            return xMid
        else: # Solution not found
            iIter += 1
            if np.sign(yMid-marketprice) == np.sign(yStart-marketprice): # If the start point and the middle point have the same sign, then the root must be in the second half of the interval   
                xStart = xMid
            else: # If the start point and the middle point have a different sign than by mean value theorem the interval must contain at least one root
                xEnd = xMid
    return "Did not converge"

In [ ]:
#cbPriced.PriceBond(coupontargetrates,notionaltargetrates,coupondatefrac,notionaldatefrac,cbPriced.couponcfs,cbPriced.notionalcfs,cbPriced.sspread,np.array([0.01,0.01]))

# STIL TO UPDATE FROM HERE DOWNWARD

In [22]:
#cbPriced = CorporateBondPriced(MD,compounding)

In [23]:
#[coupondatesconsidered,coupondatefrac] = cbPriced.refactordates(cb.coupondates,MD)
#[notionaldatesconsidered,notionaldatefrac] = cbPriced.refactordates(cb.notionaldates,MD)

In [24]:
# carry forward the sspread of bonds that are still relevant
#cbPriced.sspread = []
#cbPriced.marketprice = []

#for iCount in range(0,len(notionaldatesconsidered)-1):
#    if notionaldatesconsidered[iCount] == [0]:
#        cbPriced.sspread = np.append(cbPriced.sspread,cb.sspread[iCount])
#        cbPriced.marketprice = np.append(cbPriced.marketprice,cb.marketprice[iCount])

In [25]:
#coupontargetrates = []
#notionaltargetrates = []

#for iAsset in range(0,issuedate.size-1):
#
#    if not notionaldatesconsidered:
#       print("This bond has matured") 
#    else: 
#        coupontargetrates.append(Curves.SWExtrapolate(np.transpose(coupondatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
#        notionaltargetrates.append(Curves.SWExtrapolate(np.transpose(notionaldatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
#        cbPriced.coupondatefrac.append(np.transpose(coupondatefrac[iAsset]))
#        cbPriced.notionaldatefrac.append(np.transpose(notionaldatefrac[iAsset]))
#        cbPriced.couponcfs.append(cb.couponcfs[iAsset][coupondatesconsidered[iAsset]])
#        cbPriced.notionalcfs.append(cb.notionalcfs[iAsset])

IndexError: list index out of range

Remove cashflows that are not considered

In [ ]:
#cbPriced.PriceBond(coupontargetrates,notionaltargetrates,coupondatefrac,notionaldatefrac,cbPriced.couponcfs,cbPriced.notionalcfs,cbPriced.sspread,np.array([0.01,0.01]))

### Calibration of two hypothetical bonds that mature after modelling date

In [ ]:
def BisectionsSpread(obj, xStart, xEnd, coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread, marketprice, Precision, maxIter):
    """
    Bisection root finding algorithm for finding the root of a function. The function here is the allowed difference between the ultimate forward rate and the extrapolated curve using Smith & Wilson.

    Args:
        cbPriced =  CorporateBondPriced object containing the list of priced bonds, spreads and cash flows
        xStart =    1 x 1 floating number representing the minimum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.05
        xEnd =      1 x 1 floating number representing the maximum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.8
        M_Obs =     n x 1 ndarray of maturities of bonds, that have rates provided in input (r). Ex. u = [[1], [3]]
        r_Obs =     n x 1 ndarray of rates, for which you wish to calibrate the algorithm. Each rate belongs to an observable Zero-Coupon Bond with a known maturity. Ex. r = [[0.0024], [0.0034]]
        ufr  =      1 x 1 floating number, representing the ultimate forward rate. Ex. ufr = 0.042
        Tau =       1 x 1 floating number representing the allowed difference between ufr and actual curve. Ex. Tau = 0.00001
        Precision = 1 x 1 floating number representing the precision of the calculation. Higher the precision, more accurate the estimation of the root
        maxIter =   1 x 1 positive integer representing the maximum number of iterations allowed. This is to prevent an infinite loop in case the method does not converge to a solution         
    
    Returns:
        1 x 1 floating number representing the optimal value of the parameter alpha 

    Example of use:
        >>> import numpy as np
        >>> from SWCalibrate import SWCalibrate as SWCalibrate
        >>> M_Obs = np.transpose(np.array([1, 2, 4, 5, 6, 7]))
        >>> r_Obs =  np.transpose(np.array([0.01, 0.02, 0.03, 0.032, 0.035, 0.04]))
        >>> xStart = 0.05
        >>> xEnd = 0.5
        >>> maxIter = 1000
        >>> alfa = 0.15
        >>> ufr = 0.042
        >>> Precision = 0.0000000001
        >>> Tau = 0.0001
        >>> BisectionAlpha(xStart, xEnd, M_Obs, r_Obs, ufr, Tau, Precision, maxIter)
        [Out] 0.11549789285636511

    For more information see https://www.eiopa.europa.eu/sites/default/files/risk_free_interest_rate/12092019-technical_documentation.pdf and https://en.wikipedia.org/wiki/Bisection_method
    
    Implemented by Gregor Fabjan from Qnity Consultants on 17/12/2021.
    """   

    yStart = obj.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xStart)
    yEnd = obj.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xEnd)
    if np.abs(yStart-marketprice) < Precision:
        return xStart
    if np.abs(yEnd-marketprice) < Precision:
        return xEnd # If final point already satisfies the conditions return end point
    iIter = 0
    while iIter <= maxIter:
        xMid = (xEnd+xStart)/2 # calculate mid-point
        yMid = obj.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xMid) # What is the solution at midpoint
        if ((yStart-marketprice) == 0 or (xEnd-xStart)/2 < Precision): # Solution found
            return xMid
        else: # Solution not found
            iIter += 1
            if np.sign(yMid-marketprice) == np.sign(yStart-marketprice): # If the start point and the middle point have the same sign, then the root must be in the second half of the interval   
                xStart = xMid
            else: # If the start point and the middle point have a different sign than by mean value theorem the interval must contain at least one root
                xEnd = xMid
    return "Did not converge"

In [ ]:
cbPriced.zspread = []
for iCount in range(0,len(coupontargetrates)):
    zSpreadTmp = BisectionsSpread(cbPriced, -0.2, 0.2, coupontargetrates[iCount],notionaltargetrates[iCount],cbPriced.coupondatefrac[iCount],cbPriced.notionaldatefrac[iCount],cbPriced.couponcfs[iCount],cbPriced.notionalcfs[iCount],cbPriced.sspread[iCount],cbPriced.marketprice[iCount], Precision, 1000)
    cbPriced.zspread = np.append(cbPriced.zspread,zSpreadTmp)

In [ ]:
cbPriced.zspread

array([0.00056424, 0.02093499])

In [ ]:
cbPriced.marketprice

array([94., 92.])

In [ ]:
coupontargetrates

[array([0.04474545, 0.04352242, 0.04174698, 0.04015462]),
 array([0.04474545, 0.04352242, 0.04174698, 0.04015462, 0.03903412,
        0.03830698])]

In [ ]:
cbPriced.sspread

array([0.01, 0.01])

In [ ]:
cbPriced.zspread

array([0.00056424, 0.02093499])